In [6]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os


train_dir = r"C:\Users\admin\Documents\mini_proj_bird\masked_images\train\train"
valid_dir = r"C:\Users\admin\Documents\mini_proj_bird\masked_images\valid\valid"  
test_dir = r"C:\Users\admin\Documents\mini_proj_bird\masked_images\test\test"   

# Image size and batch size
img_size = (224, 224)
batch_size = 32

# Define ImageDataGenerators with data augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# For validation and test sets, only rescaling is needed
valid_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Load images from the directories
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',  
    shuffle=True
)

valid_generator = valid_datagen.flow_from_directory(
    valid_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',  
    shuffle=False
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',  
    shuffle=False
)


Found 7045 images belonging to 200 classes.
Found 2369 images belonging to 200 classes.
Found 2374 images belonging to 200 classes.


In [8]:
resnet_model=tf.keras.applications.InceptionResNetV2(input_shape=(224,224,3),
                                               include_top=False,
                                               weights='imagenet')
resnet_model.trainable = False


model_resnet = keras.Sequential([  
    resnet_model,
    layers.Flatten(),
    layers.Dense(units=1950,activation='relu'),
    layers.BatchNormalization(),
    layers.Dense(units=200, activation="softmax"),
])
model_resnet.summary()



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_resnet_v2 (Funct  (None, 5, 5, 1536)        54336736  
 ional)                                                          
                                                                 
 flatten (Flatten)           (None, 38400)             0         
                                                                 
 dense (Dense)               (None, 1950)              74881950  
                                                                 
 batch_normalization_609 (B  (None, 1950)              7800      
 atchNormalization)                                              
                                                                 
 dense_1 (Dense)             (None, 200)               390200    
                                                                 
Total params: 129616686 (494.45 MB)
Trainable params: 75

In [9]:
model_resnet.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)



In [10]:
#fit the model

from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(monitor='val_loss',patience=10)

history = model_resnet.fit(
    train_generator,
    validation_data=valid_generator,
    epochs=80,
    verbose=1,
    callbacks=[early_stop]
)



Epoch 1/80
221/221 [==============================] - 590s 3s/step - loss: 4.3773 - accuracy: 0.1509 - val_loss: 4.1710 - val_accuracy: 0.1984
Epoch 2/80
221/221 [==============================] - 676s 3s/step - loss: 3.0531 - accuracy: 0.2840 - val_loss: 3.4810 - val_accuracy: 0.2727
Epoch 3/80
221/221 [==============================] - 781s 4s/step - loss: 2.5536 - accuracy: 0.3705 - val_loss: 2.8782 - val_accuracy: 0.3385
Epoch 4/80
221/221 [==============================] - 674s 3s/step - loss: 2.2049 - accuracy: 0.4277 - val_loss: 3.0034 - val_accuracy: 0.3322
Epoch 5/80
221/221 [==============================] - 745s 3s/step - loss: 1.9977 - accuracy: 0.4634 - val_loss: 2.6846 - val_accuracy: 0.3829
Epoch 6/80
221/221 [==============================] - 577s 3s/step - loss: 1.8058 - accuracy: 0.5075 - val_loss: 2.5480 - val_accuracy: 0.4183
Epoch 7/80
221/221 [==============================] - 570s 3s/step - loss: 1.6328 - accuracy: 0.5394 - val_loss: 2.5721 - val_accuracy: 0.4073

In [11]:
model_resnet.evaluate(test_generator)

75/75 [==============================] - 117s 2s/step - loss: 2.4364 - accuracy: 0.4848


[2.436426877975464, 0.4848357141017914]

In [12]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

# Predict class probabilities
predictions = model_resnet.predict(test_generator)
y_pred = np.argmax(predictions, axis=1)
y_true = test_generator.classes  # True labels from the generator

# Class labels
class_labels = list(test_generator.class_indices.keys())

# Generate classification report
print("Classification Report:")
print(classification_report(y_true, y_pred, target_names=class_labels))

# Generate confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(y_true, y_pred))


75/75 [==============================] - 106s 1s/step
Classification Report:
                                    precision    recall  f1-score   support

        001.Black_footed_Albatross       0.62      0.42      0.50        12
              002.Laysan_Albatross       0.33      0.58      0.42        12
               003.Sooty_Albatross       0.40      0.33      0.36        12
             004.Groove_billed_Ani       0.78      0.58      0.67        12
                005.Crested_Auklet       1.00      0.56      0.71         9
                  006.Least_Auklet       1.00      0.78      0.88         9
               007.Parakeet_Auklet       0.77      0.91      0.83        11
             008.Rhinoceros_Auklet       0.50      0.30      0.37        10
              009.Brewer_Blackbird       0.33      0.17      0.22        12
          010.Red_winged_Blackbird       0.78      0.58      0.67        12
               011.Rusty_Blackbird       0.20      0.08      0.12        12
       012

In [16]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Load an image for testing
image_path = r"C:\Users\admin\Documents\mini_proj_bird\masked_images\train\train\006.Least_Auklet\Least_Auklet_0008_795071.jpg"
img = load_img(image_path, target_size=(224, 224))
img_array = img_to_array(img) / 255.0  # Normalize the image
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

# Predict the class
pred = model_resnet.predict(img_array)
predicted_class = np.argmax(pred)
predicted_label = class_labels[predicted_class]

print(f"Predicted Class: {predicted_label}")


1/1 [==============================] - 0s 397ms/step
Predicted Class: 006.Least_Auklet


In [14]:
model_resnet.save("model_resnet")  


INFO:tensorflow:Assets written to: model_resnet\assets


INFO:tensorflow:Assets written to: model_resnet\assets


In [17]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Load an image for testing
image_path = r"C:\Users\admin\Documents\mini_proj_bird\masked_images\train\train\021.Eastern_Towhee\Eastern_Towhee_0007_22172.jpg"
img = load_img(image_path, target_size=(224, 224))
img_array = img_to_array(img) / 255.0  # Normalize the image
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

# Predict the class
pred = model_resnet.predict(img_array)
predicted_class = np.argmax(pred)
predicted_label = class_labels[predicted_class]

print(f"Predicted Class: {predicted_label}")


1/1 [==============================] - 0s 453ms/step
Predicted Class: 021.Eastern_Towhee


In [18]:
print(f"Train accuracy: {history.history['accuracy'][-1]:.4f}")
print(f"Test accuracy: {history.history['val_accuracy'][-1]:.4f}")
print(f"Train loss: {history.history['loss'][-1]:.4f}")
print(f"Test loss: {history.history['val_loss'][-1]:.4f}")

Train accuracy: 0.7896
Test accuracy: 0.4795
Train loss: 0.6930
Test loss: 2.4766
